In [1]:
import sys
print(sys.version)

3.8.2 | packaged by conda-forge | (default, Apr 24 2020, 08:20:52) 
[GCC 7.3.0]


In [ ]:
import pytorch_lightning
pytorch_lightning.__version__

In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('../../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import numpy as np
import torch
import pytorch_lightning as pl
from pytorch_lightning import loggers
import config


from classification.models.M5 import M5, M5PLModule
from classification.models.SpectrogramCNN import SpectrogramCNNPLModule

from utils.RobustnessExperiment import load_module
from utils.Visual import show_audio, show_spectogram, show_waveplot

/nfs/homedirs/herrmanp/miniconda3/envs/ml/lib/python3.8/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/nfs/homedirs/herrmanp/miniconda3/envs/ml/lib/python3.8/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [4]:
from datasets.datasethandler import DatasetHandler
datasetHandler = DatasetHandler()

# Train Model

In [ ]:
from pytorch_lightning.callbacks import Callback

class SaveCallback(Callback): 
    def __init__(self, model_name = "newest_model"):
        super().__init__()
        self.model_name = model_name
        self.best_val_acc = None

    def on_epoch_end(self, trainer, pl_module):
        if not self.best_val_acc or pl_module.val_results_history[-1]["val_acc"] > self.best_val_acc:
            print("new best val acc", pl_module.val_results_history[-1]["val_acc"])
            self.best_val_acc = pl_module.val_results_history[-1]["val_acc"]
            save_path = self.model_name + "_v{}".format(trainer.logger.version) +  "_best.p"
            pl_module.save(save_path)
            print("Saved checkpoint at epoch {} at \"{}\"".format((trainer.current_epoch + 1), save_path))

cb = SaveCallback("iiii")

In [ ]:
# best M5: batchsize 32, lr 9e-4, weithgdecay 0.001, lr decay 0.85
import config
import pytorch_lightning as pl
from classification.models.M5 import M5, M5PLModule
from classification.models.SpectrogramCNN import SpectrogramCNNPLModule
from attacks.pgd import ProjectedGradientDescent

print("Currently Best:", cb.best_val_acc)

hparams = {
    "batch_size": 32,
    "learning_rate": 9e-4,
    "weight_decay": 1e-3,
    "lr_decay": 0.85,
    "epochs": 100
}

model = M5PLModule(hparams)
model.prepare_data()
datasetHandler.load(model, 'training')
datasetHandler.load(model, 'validation')

# to train with gaussian noise:
model.set_smooth(50)

trainer = pl.Trainer(
    max_epochs=hparams["epochs"],
    logger= loggers.TensorBoardLogger(config.LOG_DIR, name=type(model).__name__),
    gpus=1 if torch.cuda.is_available() else None,
    callbacks=[cb]
)

trainer.fit(model)

In [ ]:
model.save("/nfs/students/summer-term-2020/project-4/SAVED_MODELS/M5/M5_vanilla.p")

In [ ]:
!ls

In [30]:
model = load_module("optuna_spectrogram", SpectrogramCNNPLModule)
#model = load_module("/nfs/students/summer-term-2020/project-4/SAVED_MODELS/M5/M5_vanilla.p", M5PLModule)
model.prepare_data()
datasetHandler.load(model, 'training')
datasetHandler.load(model, 'validation')
model.cuda()
#model.report()

Load: /nfs/students/summer-term-2020/project-4/data/dataset1/dataset_resampled/training.p
Load: /nfs/students/summer-term-2020/project-4/data/dataset1/dataset_resampled/validation.p


SpectrogramCNNPLModule(
  (model): SpectrogramCNN(
    (bn0): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv1): Conv2d(1, 5, kernel_size=(10, 10), stride=(2, 2))
    (bn1): BatchNorm2d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(5, 5, kernel_size=(10, 10), stride=(2, 2))
    (bn2): BatchNorm2d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(5, 10, kernel_size=(20, 20), stride=(3, 3))
    (bn3): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv4): Conv2d(10, 15, kernel_size=(20, 20), stride=(3, 3))
    (bn4): BatchNorm2d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc1): Linear(in_features=5100, out_features=50, bias=True)
    (fc2): Linear(in_features=50, out_features=2, bias=True)
  )
)

### Run a single attack

In [37]:
from attacks.pgd import ProjectedGradientDescent
attack = ProjectedGradientDescent(model.model, model.val_dataloader(), {"norm":"inf", "epsilon": 2, "num_iter":2}, early_stopping=-1, device='cuda', save_samples=True)
attack.attack()
attack.report( )
attack.showAdversarialExample(target_class=1)

100%|██████████| 211/211 [00:26<00:00,  7.91it/s]


Attack-Parameters:	{'norm': 'inf', 'epsilon': 2, 'num_iter': 2}
Early stopping: 	False (-1)

Successfully attacked:	708
Total attacked: 	1508
Total processed:	1687

Success-Rate: 		0.47
Perturbed Accurracy: 	0.47


Will always strive towards 0.5, e.g. for epsilon = 2.
also for spectrogram model.

In [ ]:
attack.showAdversarialExample(target_class=1)

In [ ]:
from attacks.volume import VolumeAttack
from attacks.pitch import PitchAttack
from attacks.speed import TimeStretchAttack



attack = VolumeAttack(model.model, model.val_dataloader(), {"epsilon":0.9}, early_stopping=-1, device='cuda', save_samples=True)

attack.attack()
attack.report( )

In [ ]:
attack.showAdversarialExample(target_class=1)

* change:
    * return accuracy
    * make easy to compare via spectogram & waveform
